# Importing Necessary Libraries

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Training

## Loading and Splitting the Data

In [5]:
X = np.load("../Features.npy")
y = np.load("../Labels (Mutli Class).npy")

In [6]:
X.shape, y.shape

((109416, 250, 2), (109416,))

In [7]:
X, y = X.astype(np.float32), y.astype(np.float32)

In [8]:
X = np.expand_dims(X, axis = -1)

In [9]:
X.shape, y.shape

((109416, 250, 2, 1), (109416,))

In [10]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, shuffle = True, random_state = 42, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, shuffle = True, random_state = 42, test_size = 0.2)

In [11]:
X_train.shape, X_val.shape, X_test.shape

((70025, 250, 2, 1), (17507, 250, 2, 1), (21884, 250, 2, 1))

In [12]:
# Numberof classes
np.unique(y_train).shape

(14,)

In [13]:
X_train.dtype

dtype('float32')

## Model Definition, Compilation and Declaration

In [25]:
def build_ecg_cnn_model(input_shape = (250, 2, 1), num_classes=14):
    model = Sequential([
        # First convolution layer
        layers.Conv2D(32, (3, 3), activation='relu', padding = "same", input_shape=(250, 2, 1)),
        layers.MaxPooling2D((2, 1)),
        
        # Second convolution layer
        layers.Conv2D(64, (3, 3), activation='relu', padding = "same"),
        layers.MaxPooling2D((2, 1)),

        # Third convolution layer
        layers.Conv2D(128, (3, 3), activation='relu', padding = "same"),
        layers.MaxPooling2D((2, 1)),

        # Fourth convolution layer
        layers.Conv2D(256, (3, 3), activation='relu', padding = "same"),
        layers.MaxPooling2D((2, 1)),
        
        # Flattening and fully connected layers
        layers.Flatten(),
        
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(num_classes, activation='softmax')  # For binary classification (normal/abnormal)
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [27]:
model = build_ecg_cnn_model()

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 250, 2, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 125, 2, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 2, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 2, 64)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 62, 2, 128)        73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 2, 128)       0

## Fitting The Model

In [29]:
early_stop = EarlyStopping(monitor = "val_loss", patience = 3, restore_best_weights = True)

In [31]:
history = model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = 50, batch_size = 64)

Epoch 1/50
1095/1095 [==============================] - 17s 11ms/step - loss: 0.2430 - accuracy: 0.9413 - val_loss: 0.1123 - val_accuracy: 0.9718
Epoch 2/50
1095/1095 [==============================] - 12s 11ms/step - loss: 0.0855 - accuracy: 0.9791 - val_loss: 0.0703 - val_accuracy: 0.9818
Epoch 3/50
1095/1095 [==============================] - 12s 11ms/step - loss: 0.0628 - accuracy: 0.9838 - val_loss: 0.0622 - val_accuracy: 0.9853
Epoch 4/50
1095/1095 [==============================] - 12s 11ms/step - loss: 0.0511 - accuracy: 0.9861 - val_loss: 0.0539 - val_accuracy: 0.9866
Epoch 5/50
1095/1095 [==============================] - 12s 11ms/step - loss: 0.0445 - accuracy: 0.9880 - val_loss: 0.0487 - val_accuracy: 0.9874
Epoch 6/50
1095/1095 [==============================] - 12s 11ms/step - loss: 0.0370 - accuracy: 0.9896 - val_loss: 0.0483 - val_accuracy: 0.9870
Epoch 7/50
1095/1095 [==============================] - 12s 11ms/step - loss: 0.0353 - accuracy: 0.9900 - val_loss: 0.0442 -

## Saving The Model

In [46]:
# Saving the model in .h5 format
model.save("../Models/Model 2D.h5")

In [48]:
# Saving the model in .keras format
model.save("../Models/Model 2D.keras")

In [50]:
# Saving the model in tf format
model.save("../Model 2D", save_format = "tf")

INFO:tensorflow:Assets written to: ../Model 2D\assets


INFO:tensorflow:Assets written to: ../Model 2D\assets


# Saving the Model Training History

In [53]:
hist_df = pd.DataFrame(history.history)

In [55]:
hist_df.to_csv("../History 2D.csv")